In [1]:
import google.generativeai as genai
import pandas as pd
import time

In [ ]:
# Necesitaras crear un archivo GEMINI_API_KEYS.txt con las claves de Gemini que tengas
with open('GEMINI_API_KEY.txt') as f:
    genai.configure(api_key=f.readline())

In [3]:
MIN_LENGTH = 251
MAX_LENGTH = 300

In [4]:
df = pd.read_excel('DESCRIPCIONES_TODAS.xlsx')
df = df.dropna(subset='Código de la especie (id taxon)')
df['Código de la especie (id taxon)'] = df['Código de la especie (id taxon)'].astype(int)
df = df[~df.duplicated(subset='Código de la especie (id taxon)')]
df = df[(df['Descripción'].str.len() < MAX_LENGTH) & (df['Descripción'].str.len() >= MIN_LENGTH)].copy().reset_index(drop=True)
df

,Nombres comunes,Nombre aceptado,Fuente,Grupo taxonómico,Descripción,Código de la especie (id taxon)
0,Aratinga ñanday,Aratinga nenday,SEO BirdLife,Aves,"La aratinga ñanday es en su mayoría verde, con...",36640
1,Correlimos batitú,Bartramia longicauda,SEO BirdLife,Aves,El correlimos batitú tiene el tamaño del archi...,16348
2,Gallineta chica,Paragallinula angulata,SEO BirdLife,Aves,"La gallineta chica se parece a la común, pero ...",80918
3,Garza azulada,Ardea herodias,SEO BirdLife,Aves,La garza azulada es una ardeida de gran tamaño...,16286
4,Paíño de Swinhoe,Hydrobates monorhis,SEO BirdLife,Aves,El paíño de Swinhoe presenta la cola ligeramen...,80924
...,...,...,...,...,...,...
150,Lagarto gigante de El Hierro,Gallotia simonyi,VertebradosIbericos - Reptiles,Reptiles,"Lagarto de talla grande, de color pardo negruz...",12219
151,Lagarto tizón,Gallotia galloti,VertebradosIbericos - Reptiles,Reptiles,"Lagarto de talla mediana, de color pardo negru...",11338
152,Lagarto canario moteado,Gallotia intermedia,VertebradosIbericos - Reptiles,Reptiles,Lagarto de talla mediana con escamas temporale...,11516
153,Lagarto gigante de La Palma,Gallotia auaritae Mateo,VertebradosIbericos - Reptiles,Reptiles,Al tratarse de una especie para la que únicame...,10796


In [ ]:
model = genai.GenerativeModel('gemini-2.0-flash')
df['Descripción Gemini 1'] = ''
df['Descripción Gemini 2'] = ''
df['Descripción Gemini 3'] = ''
for i, row in df.iterrows():
    if (i+1) % 15 == 0:
        print(f'Starting prompt {i+1}...', end='\r')
        time.sleep(60)
    messages = [
        {'role': 'user',
        'parts': ["""Quiero fabricar descripciones de especies que una persona común sin conocimientos técnicos pueda usar para identificar las especies en la naturaleza.
        Convierte la descripción que te voy a dar a continuación en una descripción más sencilla, que sea de aproximadamente 100 palabras.
        No incluyas información que no esté en la descripción original, y devuelve solamente la descripción simplificada.
        Puedes omitir información que no sea tan relevante para la identificación de una persona común (que no sabe nombres científicos) y cambiar la estructura.
        También puedes unir frases si así se consigue una lectura más fluida y sin tantas oraciones separadas."""]},
        {'role': 'model',
        'parts': ["Entendido."]},
        {'role': 'user',
        'parts': [row['Descripción']]}
    ]
    response = model.generate_content(messages, generation_config=genai.types.GenerationConfig(candidate_count=3))
    descriptions = [c.content.parts[0].text.strip() for c in response.candidates]
    df.loc[i, 'Descripción Gemini 1'] = descriptions[0]
    df.loc[i, 'Descripción Gemini 2'] = descriptions[1]
    df.loc[i, 'Descripción Gemini 3'] = descriptions[2]
print(f'Finished all {len(df)} prompts')
df.rename(columns={'Descripción': 'Descripción Original'}).to_excel('Descripciones_Gemini_Cortas.xlsx', index=False)

Finished all 155 prompts
